# Detection

In [5]:
# common imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import matplotlib.pyplot as plt
import mxnet as mx
import numpy as np

In [6]:
# Camera imports
from picamera import PiCamera
from PIL import Image
import io

In [7]:
# SSD imports
import gluoncv as gcv

from scipy import ndimage

from mxnet import nd
from gluoncv.model_zoo import get_model

In [8]:
# MTCNN imports
#import cv2
import sys
import os
from scipy import misc
import random

from easydict import EasyDict as edict
from ArcFace.mtcnn_detector import MtcnnDetector

from mxnet.contrib.onnx.onnx2mx.import_model import import_model


## Face Detetion

In [9]:
# Download model
for i in range(4):
    mx.test_utils.download(dirname='ArcFace/mtcnn-model', url='https://s3.amazonaws.com/onnx-model-zoo/arcface/mtcnn-model/det{}-0001.params'.format(i+1))
    mx.test_utils.download(dirname='ArcFace/mtcnn-model', url='https://s3.amazonaws.com/onnx-model-zoo/arcface/mtcnn-model/det{}-symbol.json'.format(i+1))

# Determine and set context
ctx = mx.cpu()
det_threshold = [0.6,0.7,0.8]
mtcnn_path = os.path.join(os.path.dirname('__file__'), 'ArcFace/mtcnn-model')
detector = MtcnnDetector(model_folder=mtcnn_path, ctx=ctx, num_worker=1, accurate_landmark = False, threshold=det_threshold)

### Testing loop

In [10]:
from Robot import Robot

In [11]:
robot = Robot()

('Report logged', 'SERVO::Move channel 0 to 50 percent. Value of 375')
('Report logged', 'SERVO::Move channel 1 to 50 percent. Value of 375')
('Report logged', 'SERVO::Move channel 2 to 10 percent. Value of 195')
('Report logged', 'SERVO::Move channel 3 to 57 percent. Value of 406')
('Report logged', 'SERVO::Move channel 4 to 70 percent. Value of 465')
('Report logged', 'SERVO::Move channel 5 to 10 percent. Value of 195')


In [12]:
import time, threading

In [36]:
%time
print(time.time())
move_robot(robot, robot.LEFT_RIGHT, -0.4)

CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 31.9 µs
1537448803.0
New value 43.7138972673
('Report logged', 'SERVO::Move channel 0 to 27.0243082306 percent. Value of 271')
('Report logged', 'SERVO::Move channel 0 to 27.8587876825 percent. Value of 275')
('Report logged', 'SERVO::Move channel 0 to 28.6932671343 percent. Value of 279')
('Report logged', 'SERVO::Move channel 0 to 29.5277465861 percent. Value of 282')
('Report logged', 'SERVO::Move channel 0 to 30.3622260379 percent. Value of 286')
('Report logged', 'SERVO::Move channel 0 to 31.1967054898 percent. Value of 290')
('Report logged', 'SERVO::Move channel 0 to 32.0311849416 percent. Value of 294')
('Report logged', 'SERVO::Move channel 0 to 32.8656643934 percent. Value of 297')
('Report logged', 'SERVO::Move channel 0 to 33.7001438453 percent. Value of 301')
('Report logged', 'SERVO::Move channel 0 to 34.5346232971 percent. Value of 305')
('Report logged', 'SERVO::Move channel 0 to 35.3691027489 percent. Value of 3

In [4]:
with PiCamera() as camera:
    #camera.rotation = 180
    camera.resolution = (341, 256)    
    time.sleep(1)
    stream = io.BytesIO()
    for i, _ in enumerate(camera.capture_continuous(stream, format='jpeg', burst=True)):
        tic = time.time()
        image = np.array(Image.open(stream))
        stream.truncate()
        stream.seek(0)
        stream.flush()
        output = detector.detect_face(image)
        if output is not None:
            output = output[0][0].tolist()
            if output[4] > 0.95:
                x1, y1, x2, y2, score = output
                plt.plot([x1, x2], [y1, y2], "o")
                x1 /= max(0, camera.resolution[0])
                x2 /= max(0, camera.resolution[0])
                y1 /= max(0, camera.resolution[1])
                y2 /= max(0, camera.resolution[1])
                
                area_ratio = (x2-x1)*(y2-y1) - 0.25
                clock_turn = (x2 + x1)/2 - 0.5
                high_low = (y2 + y1)/2 - 0.5
                
                move_robot(robot, robot.LEFT_RIGHT, clock_turn)
                move_robot(robot, robot.UP_DOWN, high_low)
                move_robot(robot, robot.CLOSE_FAR, area_ratio)
                
                plt.imshow(image)
                plt.show()
        if i > 20:
            break

NameError: name 'PiCamera' is not defined

In [24]:
left_right

-0.23739798362653625